In [1]:
from pylibs.data_processing import ranking
from pylibs.predictor import Predictor
from pylibs.text_utils import levenshtein_distance_weighted
from recognizer import ProductCodeRecognizer
from pylibs import via_utils

/home/agazade/.conda/envs/main/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
Using TensorFlow backend.


In [2]:
dataset_dir_path = "/srv/data_science/storage/product_code_ocr"
noise_df_name = "noise_processed"
relabel_df_name = "relabel"
ideal_df_name = 'ideal'
image_folder = "images_train"
max_num_to_score = 30000
total_size = 10000
batch_size = 128
bad_quantiles = (0, 0.3)
normal_quantiles = (0.3, 0.8)
good_quantiles = (0.8, 1)
bad_percent = 30
normal_percent = 20
good_percent = 50
split_dict = {
    "bounds": {
        "bad": bad_quantiles,
        "normal": normal_quantiles,
        "good": good_quantiles,
    },
    "percentages": {"bad": bad_percent, "normal": normal_percent, "good": good_percent},
}


def rel_leven_metric(machine, manual):
    return 1 - levenshtein_distance_weighted(machine, manual, 1, 2, 1) / len(manual)

metric_function = rel_leven_metric
score_column = "text"

In [3]:
model_path = '/srv/data_science/training/checkpoints/product_code_ocr/model3/epoch_80_val_loss0.052070.hdf5'
predictor = ProductCodeRecognizer(model_path)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
ranking.split_train_dataset(
    dataset_dir_path,
    noise_df_name,
    relabel_df_name,
    ideal_df_name,
    image_folder,
    max_num_to_score,
    total_size,
    batch_size,
    predictor,
    split_dict,
    metric_function,
    score_column,
)

Bad image: /srv/data_science/storage/product_code_ocr/images_train/2/c/8/2c8fab5d7c957f6be00997e7750e7c76049bea32.jpg
-2.0 1.0
1.0 1.0
1.0 1.0
Part bad. Size 3000
Part normal. Size 0
Part good. Size 0
Relabel df size: 3000
Ideal df size: 0


In [11]:
dataset_dir_path = '/srv/data_science/storage/product_code_ocr'
df_name = 'relabel'
image_dir_path = '/srv/data_science/storage/product_code_ocr/processed_images'
task_name = 'task2'
save_dir = f'/srv/data_science/storage/product_code_ocr/labeling_tasks/{task_name}'
file_attributes_key = ('Код продукта', )
df_column_names = ('text', )
divide_size = 3000
values_from_json = False
image_save_dir = f'/opt/share-http/{task_name}'
photo_id_sharding = False
img_url_format = 'http://10.0.3.2:2000/%s/{tag_id}.jpg' % task_name

In [12]:
via_utils.export_labels_to_via(
    dataset_dir_path,
    df_name,
    image_dir_path,
    save_dir,
    file_attributes_key,
    df_column_names,
    divide_size,
    values_from_json=values_from_json,
    img_url_format=img_url_format,
    image_save_dir=image_save_dir,
    photo_id_sharding=photo_id_sharding,
)

In [2]:
dataset_dir_path = '/srv/data_science/storage/product_code_ocr/labeling_results/result2'
file_attributes_key = ('Код продукта', )
df_column_names = ('text', )
save_df_name = 'relabel_via'

In [3]:
via_utils.import_labels_from_via(dataset_dir_path, file_attributes_key, df_column_names, save_df_name)